# Fine tune GPT-4o-mini for classification


In [7]:
!pip install openai --quiet

In [4]:
SYSTEM_PROMPT = """You are a classifier. Given a question, you need to classify it into one of the following classes:\n - CANCEL\n - INVOICE\n - DELIVERY\n - FEEDBACK\n - CONTACT\n - ORDER\n - REFUND\n - SHIPPING\n - SUBSCRIPTION\n - PAYMENT\n - ACCOUNT\n\nDo not try to answer the question."""

In [5]:
# read train.csv and loop over the rows and take the question column, and the class_name column
# and create a list of objects that look like this:
# {
#         "messages": [
#             {"role": "system", "content":SYSTEM_PROMPT},
#             {"role": "user", "content": << column question>>},
#             {"role": "assistant",
#              "content": "<< column class_name >>"}
#         ]
# }
import pandas as pd
df = pd.read_csv('train.csv')

training_data = []
for i, row in df.iterrows():
    training_data.append({
        "messages": [
            {"role": "system", "content":SYSTEM_PROMPT},
            {"role": "user", "content": row['question']},
            {"role": "assistant",
             "content": row['class_name']}
        ]
    })

    

In [9]:
import json
TRAIN_OPENAI_FILE = "train-openai.jsonl"
with open(
        TRAIN_OPENAI_FILE, "w", encoding="utf-8"
) as file:
    for record in training_data:
        json_line = json.dumps(record)
        file.write(json_line + "\n")

In [16]:
import openai

client = openai.OpenAI()
response = client.files.create(
    file=open(TRAIN_OPENAI_FILE, "rb"),
    purpose="fine-tune"
)
file_id = response.id
response = client.fine_tuning.jobs.create(
    training_file=file_id,
    model="gpt-4o-mini-2024-07-18"
)
response


FineTuningJob(id='ftjob-GEmrbC7SyWR0xGV1NmIWv8dK', created_at=1725548902, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-bBHdSNHiFHSbkdpwXvV7ClOQ', result_files=[], seed=887280913, status='validating_files', trained_tokens=None, training_file='file-HUh9uTxZ1e0nXZ3rqpZ1t0Yr', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [30]:

model_id = "ft:gpt-4o-mini-2024-07-18:drift::A48JGLgp"

# Function to make a request to your fine-tuned model
def call_finetuned_model(prompt):
    try:
        # Call the completion endpoint with your fine-tuned model
        response = client.chat.completions.create(
            model=model_id,  # Use the fine-tuned model ID
            messages=[    
                {"role":"system", "content": SYSTEM_PROMPT},
                {"role":"user", "content": prompt}
            ]
        )
        # Extract and return the generated text
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        return f"Error: {str(e)}"



In [32]:
# for each row in the test.csv file, take the question column and call the function call_finetuned_model
# with the question as input and store the output in a new column called predicted_class_name
df_test = pd.read_csv('test.csv')
df_test['predicted_class_name'] = df_test['question'].apply(call_finetuned_model)
df_test

,class_name,question,id,predicted_class_name
0,CANCEL,i do not know how i can see the cancellation c...,3329,CANCEL
1,SUBSCRIPTION,help me cancelling my corporate newsletter sub...,17714,SUBSCRIPTION
2,REFUND,I paid {{Currency Symbol}}{{Refund Amount}} fo...,16152,REFUND
3,CANCEL,i want assistance checking the termination pen...,3648,CANCEL
4,REFUND,i expect a rebate of {{Currency Symbol}}{{Refu...,26431,REFUND
...,...,...,...,...
325,CONTACT,"I want to speak with customer service, helpme",8258,CONTACT
326,INVOICE,get invoice #85632,15244,INVOICE
327,FEEDBACK,help me filing a complaint,7352,FEEDBACK
328,DELIVERY,want help seeing how long it takes for my ship...,13018,DELIVERY


In [35]:
def calculate_accuracy(df):
    # use the huggingface evaluate library to evaluate the model by taking the columns
    # 'predicted_class_name' and 'class_name' as input andd calculate the accuracy
    from datasets import load_metric
    # Create a mapping of class names to numerical labels
    unique_classes = set(df['predicted_class_name']).union(set(df['class_name']))
    class_to_int = {cls_name: idx for idx, cls_name in enumerate(unique_classes)}
    
    # Map the class names to integers
    df['predicted_class_numeric'] = df['predicted_class_name'].map(class_to_int)
    df['class_numeric'] = df['class_name'].map(class_to_int)
    
    # Compute the accuracy
    metric = load_metric("accuracy")
    accuracy = metric.compute(predictions=df['predicted_class_numeric'], references=df['class_numeric'])
    
    print(f"Accuracy: {accuracy['accuracy']}")
    return accuracy

calculate_accuracy(df_test)

/var/folders/sj/3t0z94yj4m12rfbjsb8x5j5r0000gn/T/ipykernel_5929/1552406827.py:14: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


Accuracy: 0.996969696969697


{'accuracy': 0.996969696969697}